## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.5 ## EPSILON
lr_rl = 0.9 ## LEARNING RATE RL
disc_fac = 0.999 ## DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = model_name + str(epsilon) + "_"##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (7938, 15)
ds_val.shape (1641, 15)
ds_test.shape (1072, 15)
ds_rl.shape (2000, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 7938, Val 1641


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (7938, 15)
shape ds_val  : (1641, 15)
shape ds_test : (1072, 15)
shape ds_rl : (2000, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1641, 5)
shape x_train_query: (7938, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1641,)
shape x_train_tree: (7938,)
----------y_data------------
shape y_val  : (1641,)
shape y_train: (7938,)

----------TEST DATA----------
shape x_test_tree : (1072,)
shape x_test_query: (1072, 5)
shape y_test      : (1072,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1353, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.687920
1         1.678266
2         1.109980
3         1.858232
4         1.598426
           ...    
1067    107.000000
1068    101.000000
1069    123.000000
1070    120.000000
1071    138.003726
Name: time, Length: 1072, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [12.682406425476074,
  4.89743709564209,
  14.758463859558105,
  14.758463859558105,
  17.954404830932617,
  1.5937821865081787,
  14.758463859558105,
  1.5922574996948242,
  37.32883071899414,
  14.758463859558105,
  14.758463859558105,
  53.39189147949219,
  2.7885539531707764,
  14.758463859558105,
  14.758463859558105,
  14.758463859558105,
  44.663673400878906,
  4.89743709564209,
  1.6255521774291992,
  1.5004899501800537,
  14.758463859558105,
  1.5937821865081787,
  27.82819366455078,
  1.5922574996948242,
  36.944313049316406,
  13.64535903930664,
  2.79833984375,
  1.6255521774291992,
  37.06562042236328,
  14.758463859558105,
  14.758463859558105,
  84.42827606201172,
  1.5922574996948242,
  14.758463859558105,
  3.980473518371582,
  14.758463859558105,
  13.64535903930664,
  10.707067489624023,
  2.7885539531707764,
  4.35945987701416,
  6.078582763671875,
  39.093868255615234,
  2.81247878074646,
  14.762251853942871,
  24.007583618164062,
  14.758463859558105,
  

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (2000, 67)
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
iteration = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 2000


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-03 02:22:14.965970
2022-01-03 02:23:01.744040 : 10/2000, index 9 evaluating
2022-01-03 02:23:56.991516 : 20/2000, index 19 evaluating
2022-01-03 02:24:51.759160 : 30/2000, index 29 evaluating
2022-01-03 02:25:44.174381 : 40/2000, index 39 evaluating
2022-01-03 02:26:37.876220 : 50/2000, index 49 evaluating
2022-01-03 02:27:36.097039 : 60/2000, index 59 evaluating
2022-01-03 02:28:20.460092 : 70/2000, index 69 evaluating
2022-01-03 02:29:20.016151 : 80/2000, index 79 evaluating
2022-01-03 02:30:09.843414 : 90/2000, index 89 evaluating
2022-01-03 02:31:02.982629 : 100/2000, index 99 evaluating
2022-01-03 02:32:08.334294 : 110/2000, index 109 evaluating
2022-01-03 02:33:11.738005 : 120/2000, index 119 evaluating
2022-01-03 02:34:08.183062 : 130/2000, index 129 evaluating
2022-01-03 02:34:58.996683 : 140/2000, index 139 evaluating
2022-01-03 02:35:53.619051 : 150/2000, index 149 evaluating
2022-01-03 02:36:43.768315 : 160/2000, index 159 evaluating
2022-01-03 02:37:45.218504

2022-01-03 04:27:19.572246 : 1370/2000, index 1369 evaluating
2022-01-03 04:28:18.275249 : 1380/2000, index 1379 evaluating
2022-01-03 04:29:14.723007 : 1390/2000, index 1389 evaluating
2022-01-03 04:30:23.517874 : 1400/2000, index 1399 evaluating
2022-01-03 04:31:25.935484 : 1410/2000, index 1409 evaluating
2022-01-03 04:32:17.491110 : 1420/2000, index 1419 evaluating
2022-01-03 04:33:13.220882 : 1430/2000, index 1429 evaluating
2022-01-03 04:34:12.406952 : 1440/2000, index 1439 evaluating
2022-01-03 04:35:08.132200 : 1450/2000, index 1449 evaluating
2022-01-03 04:36:31.239504 : 1460/2000, index 1459 evaluating
2022-01-03 04:37:29.602487 : 1470/2000, index 1469 evaluating
2022-01-03 04:38:31.425506 : 1480/2000, index 1479 evaluating
2022-01-03 04:39:29.804164 : 1490/2000, index 1489 evaluating
2022-01-03 04:40:26.759652 : 1500/2000, index 1499 evaluating
2022-01-03 04:41:22.250128 : 1510/2000, index 1509 evaluating
2022-01-03 04:42:17.689036 : 1520/2000, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_iteration = ql_df[iteration]
ql_df_iteration.to_csv(URL + 'rl_csvs/ql_df_iteration'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.178223,78.702209,49.0,49.0,51.526881,882.221248,7.178223,29.702209,7.178223,29.702209,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.277279,81.517166,49.0,49.0,52.958788,1057.366094,7.277279,32.517166,7.277279,32.517166,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.211250,79.060028,49.0,49.0,52.002131,903.605288,7.211250,30.060028,7.211250,30.060028,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.222927,79.307755,49.0,49.0,52.170676,918.559984,7.222927,30.307755,7.222927,30.307755,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
4,0,4,56.251011,81.081017,49.0,49.0,52.577159,1029.191622,7.251011,32.081017,7.251011,32.081017,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.707748,30.897945,124.0,124.0,151.099449,8667.992570,12.292252,93.102055,12.292252,93.102055,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.698135,30.899832,124.0,124.0,151.335873,8667.641324,12.301865,93.100168,12.301865,93.100168,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,107.816544,30.902151,124.0,124.0,261.904262,8667.209468,16.183456,93.097849,16.183456,93.097849,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,109.798698,30.644499,124.0,124.0,201.676966,8715.249600,14.201302,93.355501,14.201302,93.355501,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.270477,56.202030,49.0,49.0,52.859840,51.869239,7.270477,7.202030,7.270477,7.202030,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.134270,59.376747,46.0,46.0,147.240502,178.937364,12.134270,13.376747,12.134270,13.376747,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.970528,60.907295,48.0,48.0,168.234587,166.598270,12.970528,12.907295,12.970528,12.907295,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.662834,111.572830,49.0,49.0,3926.630786,3915.359079,62.662834,62.572830,62.662834,62.572830,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.636017,139.433441,111.0,111.0,820.021461,808.460576,28.636017,28.433441,28.636017,28.433441,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,104.858742,59.336945,122.0,122.0,293.822734,3926.658515,17.141258,62.663055,17.141258,62.663055,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.319763,69.686295,109.0,109.0,591.450881,1545.567436,24.319763,39.313705,24.319763,39.313705,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.536057,76.379173,136.0,136.0,507.873843,3554.642979,22.536057,59.620827,22.536057,59.620827,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## RL-Neo Multival

###### Parameters

In [28]:
epsilon_neo = epsilon ## EPSILON
epoch_neo = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [29]:
values_neo = []
values_final_neo = []
c_neo = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_neo = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_neo % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_neo}/{total_buenos_idx}, index {idx} evaluating')
    ############# ALGORITMO NEO #####################3
    for i in range(0,epoch_neo):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)    
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_neo,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_neo)
        q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)
        ##############################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]
        values_neo.append(row)
    values_final_neo.append(row)
    c_neo += 1
#print("values_neo ", values_neo)
print("len(values_neo[0])",len(values_neo[0]) )
print("len(values_neo)", len(values_neo))

Start: 2022-01-03 05:31:40.138326
2022-01-03 05:32:15.956364 : 10/2000, index 9 evaluating
2022-01-03 05:32:55.445993 : 20/2000, index 19 evaluating
2022-01-03 05:33:35.188521 : 30/2000, index 29 evaluating
2022-01-03 05:34:13.728755 : 40/2000, index 39 evaluating
2022-01-03 05:34:52.869912 : 50/2000, index 49 evaluating
2022-01-03 05:35:32.564305 : 60/2000, index 59 evaluating
2022-01-03 05:36:11.370323 : 70/2000, index 69 evaluating
2022-01-03 05:36:51.016603 : 80/2000, index 79 evaluating
2022-01-03 05:37:30.005050 : 90/2000, index 89 evaluating
2022-01-03 05:38:09.380075 : 100/2000, index 99 evaluating
2022-01-03 05:38:49.590591 : 110/2000, index 109 evaluating
2022-01-03 05:39:30.091185 : 120/2000, index 119 evaluating
2022-01-03 05:40:09.667560 : 130/2000, index 129 evaluating
2022-01-03 05:40:48.938221 : 140/2000, index 139 evaluating
2022-01-03 05:41:28.279816 : 150/2000, index 149 evaluating
2022-01-03 05:42:07.399787 : 160/2000, index 159 evaluating
2022-01-03 05:42:47.398326

2022-01-03 07:01:59.672402 : 1370/2000, index 1369 evaluating
2022-01-03 07:02:39.711170 : 1380/2000, index 1379 evaluating
2022-01-03 07:03:19.410509 : 1390/2000, index 1389 evaluating
2022-01-03 07:04:00.240434 : 1400/2000, index 1399 evaluating
2022-01-03 07:04:40.357373 : 1410/2000, index 1409 evaluating
2022-01-03 07:05:19.703814 : 1420/2000, index 1419 evaluating
2022-01-03 07:05:59.375158 : 1430/2000, index 1429 evaluating
2022-01-03 07:06:39.594424 : 1440/2000, index 1439 evaluating
2022-01-03 07:07:19.506100 : 1450/2000, index 1449 evaluating
2022-01-03 07:08:01.206285 : 1460/2000, index 1459 evaluating
2022-01-03 07:08:41.107420 : 1470/2000, index 1469 evaluating
2022-01-03 07:09:21.263912 : 1480/2000, index 1479 evaluating
2022-01-03 07:10:00.957264 : 1490/2000, index 1489 evaluating
2022-01-03 07:10:40.673044 : 1500/2000, index 1499 evaluating
2022-01-03 07:11:20.545419 : 1510/2000, index 1509 evaluating
2022-01-03 07:12:00.272619 : 1520/2000, index 1519 evaluating
2022-01-

In [30]:
neo_df = pd.DataFrame(values_neo,columns=rl_columns)
neo_df_iteration = neo_df[iteration]
neo_df_iteration.to_csv(URL + 'rl_csvs/neo_df_iteration'+str(n)+'.csv',index=False)

In [31]:
neo_df_final = pd.DataFrame(values_final_neo,columns=rl_columns)
neo_df_final.to_csv(URL + 'rl_csvs/neo_df_final'+str(n)+'.csv',index=False)

In [32]:
neo_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.178169,56.219135,49.0,49.0,51.526114,52.115914,7.178169,7.219135,7.178169,7.219135,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,56.239464,56.174950,49.0,49.0,52.409836,51.479902,7.239464,7.174950,7.239464,7.174950,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.585365,56.179802,49.0,49.0,57.537767,51.549556,7.585365,7.179802,7.585365,7.179802,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.233925,56.250275,49.0,49.0,52.329669,52.566483,7.233925,7.250275,7.233925,7.250275,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.178196,56.195019,49.0,49.0,51.526497,51.768295,7.178196,7.195019,7.178196,7.195019,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,110.968742,30.509415,124.0,124.0,169.813675,8740.489545,13.031258,93.490585,13.031258,93.490585,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.667557,30.606731,124.0,124.0,152.089156,8722.302617,12.332443,93.393269,12.332443,93.393269,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.700775,31.025974,124.0,124.0,176.869382,8644.169460,13.299225,92.974026,13.299225,92.974026,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,111.315842,30.164186,124.0,124.0,160.887872,8805.159899,12.684158,93.835814,12.684158,93.835814,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [33]:
neo_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.179394,56.191067,49.0,49.0,51.543695,51.711441,7.179394,7.191067,7.179394,7.191067,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.671173,59.729153,46.0,46.0,160.558628,188.489633,12.671173,13.729153,12.671173,13.729153,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.175026,59.955967,48.0,48.0,148.231257,142.945146,12.175026,11.955967,12.175026,11.955967,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.577660,93.126808,49.0,49.0,3915.963482,1947.175199,62.577660,44.126808,62.577660,44.126808,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P885;ᶲIRI_ID;...dire..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,137.705841,82.044243,111.0,111.0,713.201947,838.435872,26.705841,28.955757,26.705841,28.955757,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1996,1996,199,101.179825,58.697758,122.0,122.0,433.479694,4007.173878,20.820175,63.302242,20.820175,63.302242,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.225708,74.004478,109.0,109.0,586.884928,1224.686528,24.225708,34.995522,24.225708,34.995522,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.845276,71.952026,136.0,136.0,521.906630,4102.142926,22.845276,64.047974,22.845276,64.047974,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Sarsa Multival

###### Parameters

In [34]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [35]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

Start: 2022-01-03 07:44:31.638564
2022-01-03 07:45:22.287224 : 10/2000, index 9 evaluating
2022-01-03 07:46:19.810443 : 20/2000, index 19 evaluating
2022-01-03 07:47:16.712788 : 30/2000, index 29 evaluating
2022-01-03 07:48:11.129635 : 40/2000, index 39 evaluating
2022-01-03 07:49:06.407221 : 50/2000, index 49 evaluating
2022-01-03 07:50:06.870270 : 60/2000, index 59 evaluating
2022-01-03 07:50:52.889634 : 70/2000, index 69 evaluating
2022-01-03 07:51:54.736181 : 80/2000, index 79 evaluating
2022-01-03 07:52:44.702444 : 90/2000, index 89 evaluating
2022-01-03 07:53:39.378925 : 100/2000, index 99 evaluating
2022-01-03 07:54:45.595570 : 110/2000, index 109 evaluating
2022-01-03 07:55:50.792378 : 120/2000, index 119 evaluating
2022-01-03 07:56:47.765058 : 130/2000, index 129 evaluating
2022-01-03 07:57:39.381607 : 140/2000, index 139 evaluating
2022-01-03 07:58:35.631724 : 150/2000, index 149 evaluating
2022-01-03 07:59:27.582031 : 160/2000, index 159 evaluating
2022-01-03 08:00:30.543934

2022-01-03 09:52:56.762797 : 1370/2000, index 1369 evaluating
2022-01-03 09:53:56.605284 : 1380/2000, index 1379 evaluating
2022-01-03 09:54:55.036106 : 1390/2000, index 1389 evaluating
2022-01-03 09:56:05.465103 : 1400/2000, index 1399 evaluating
2022-01-03 09:57:09.172777 : 1410/2000, index 1409 evaluating
2022-01-03 09:58:01.280217 : 1420/2000, index 1419 evaluating
2022-01-03 09:58:57.651291 : 1430/2000, index 1429 evaluating
2022-01-03 09:59:57.760302 : 1440/2000, index 1439 evaluating
2022-01-03 10:00:54.319766 : 1450/2000, index 1449 evaluating
2022-01-03 10:02:18.657914 : 1460/2000, index 1459 evaluating
2022-01-03 10:03:17.654886 : 1470/2000, index 1469 evaluating
2022-01-03 10:04:20.403741 : 1480/2000, index 1479 evaluating
2022-01-03 10:05:20.094696 : 1490/2000, index 1489 evaluating
2022-01-03 10:06:18.533801 : 1500/2000, index 1499 evaluating
2022-01-03 10:07:14.994266 : 1510/2000, index 1509 evaluating
2022-01-03 10:08:11.290572 : 1520/2000, index 1519 evaluating
2022-01-

In [36]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_iteration = sarsa_df[iteration]
sarsa_df_iteration.to_csv(URL + 'rl_csvs/sarsa_df_iteration'+str(n)+'.csv',index=False)

In [37]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [39]:
sarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.934292,78.187775,49.0,49.0,62.952987,851.926189,7.934292,29.187775,7.934292,29.187775,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.176449,80.441132,49.0,49.0,51.501418,988.544756,7.176449,31.441132,7.176449,31.441132,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.180840,79.303505,49.0,49.0,51.564456,918.302412,7.180840,30.303505,7.180840,30.303505,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.200256,82.078423,49.0,49.0,51.843691,1094.182038,7.200256,33.078423,7.200256,33.078423,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
4,0,4,56.177402,56.250793,49.0,49.0,51.515107,52.574006,7.177402,7.250793,7.177402,7.250793,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,110.752243,30.753601,124.0,124.0,175.503064,8694.890913,13.247757,93.246399,13.247757,93.246399,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,110.745476,30.918348,124.0,124.0,175.682413,8664.193881,13.254524,93.081652,13.254524,93.081652,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.615799,30.777571,124.0,124.0,179.136838,8690.421320,13.384201,93.222429,13.384201,93.222429,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,109.920967,30.630304,124.0,124.0,198.219167,8717.900068,14.079033,93.369696,14.079033,93.369696,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [40]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.178875,56.242657,49.0,49.0,51.536246,52.456076,7.178875,7.242657,7.178875,7.242657,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.270027,58.095539,46.0,46.0,150.553567,146.302066,12.270027,12.095539,12.270027,12.095539,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.499912,60.769238,48.0,48.0,156.247807,163.053427,12.499912,12.769238,12.499912,12.769238,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.093430,111.020386,49.0,49.0,3855.593995,3846.528248,62.093430,62.020386,62.093430,62.020386,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.192703,138.889603,111.0,111.0,794.828516,777.829937,28.192703,27.889603,28.192703,27.889603,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,104.880798,59.061089,122.0,122.0,293.067065,3961.306573,17.119202,62.938911,17.119202,62.938911,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.350571,73.463737,109.0,109.0,592.950292,1262.825953,24.350571,35.536263,24.350571,35.536263,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.827606,73.492363,136.0,136.0,521.099605,3907.204686,22.827606,62.507637,22.827606,62.507637,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Expected-Sarsa Multival

###### Parameters

In [41]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = epsilon
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [42]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-03 12:06:58.375095
2022-01-03 12:07:48.829120 : 10/2000, index 9 evaluating
2022-01-03 12:08:46.478744 : 20/2000, index 19 evaluating
2022-01-03 12:09:43.837353 : 30/2000, index 29 evaluating
2022-01-03 12:10:38.642418 : 40/2000, index 39 evaluating
2022-01-03 12:11:34.805018 : 50/2000, index 49 evaluating
2022-01-03 12:12:35.418034 : 60/2000, index 59 evaluating
2022-01-03 12:13:21.532995 : 70/2000, index 69 evaluating
2022-01-03 12:14:24.116795 : 80/2000, index 79 evaluating
2022-01-03 12:15:14.492008 : 90/2000, index 89 evaluating
2022-01-03 12:16:09.182604 : 100/2000, index 99 evaluating
2022-01-03 12:17:14.722917 : 110/2000, index 109 evaluating
2022-01-03 12:18:19.493626 : 120/2000, index 119 evaluating
2022-01-03 12:19:16.194331 : 130/2000, index 129 evaluating
2022-01-03 12:20:07.950821 : 140/2000, index 139 evaluating
2022-01-03 12:21:04.339340 : 150/2000, index 149 evaluating
2022-01-03 12:21:56.247699 : 160/2000, index 159 evaluating
2022-01-03 12:23:00.236746

2022-01-03 14:16:21.519366 : 1370/2000, index 1369 evaluating
2022-01-03 14:17:21.849748 : 1380/2000, index 1379 evaluating
2022-01-03 14:18:20.593105 : 1390/2000, index 1389 evaluating
2022-01-03 14:19:31.742287 : 1400/2000, index 1399 evaluating
2022-01-03 14:20:36.887851 : 1410/2000, index 1409 evaluating
2022-01-03 14:21:29.146231 : 1420/2000, index 1419 evaluating
2022-01-03 14:22:26.596055 : 1430/2000, index 1429 evaluating
2022-01-03 14:23:27.689982 : 1440/2000, index 1439 evaluating
2022-01-03 14:24:24.753479 : 1450/2000, index 1449 evaluating
2022-01-03 14:25:50.281390 : 1460/2000, index 1459 evaluating
2022-01-03 14:26:50.425597 : 1470/2000, index 1469 evaluating
2022-01-03 14:27:53.801505 : 1480/2000, index 1479 evaluating
2022-01-03 14:28:53.670420 : 1490/2000, index 1489 evaluating
2022-01-03 14:29:52.282294 : 1500/2000, index 1499 evaluating
2022-01-03 14:30:49.078801 : 1510/2000, index 1509 evaluating
2022-01-03 14:31:46.045058 : 1520/2000, index 1519 evaluating
2022-01-

In [43]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_iteration = exsarsa_df[iteration]
exsarsa_df_iteration.to_csv(URL + 'rl_csvs/exsarsa_df_iteration'+str(n)+'.csv',index=False)

In [44]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [45]:
exsarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.174515,78.913612,49.0,49.0,51.473662,894.824205,7.174515,29.913612,7.174515,29.913612,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.202576,80.338203,49.0,49.0,51.877096,982.082994,7.202576,31.338203,7.202576,31.338203,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.207458,80.076286,49.0,49.0,51.947458,965.735571,7.207458,31.076286,7.207458,31.076286,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.127777,78.709625,49.0,49.0,50.805206,882.661832,7.127777,29.709625,7.127777,29.709625,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
4,0,4,56.065212,56.176968,49.0,49.0,49.917224,51.508864,7.065212,7.176968,7.065212,7.176968,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.192619,30.798937,124.0,124.0,164.029000,8686.438173,12.807381,93.201063,12.807381,93.201063,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,109.741234,30.895002,124.0,124.0,203.312413,8668.540585,14.258766,93.104998,14.258766,93.104998,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.030563,30.922100,124.0,124.0,195.145160,8663.495456,13.969437,93.077900,13.969437,93.077900,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.425476,30.898623,124.0,124.0,184.267700,8667.866490,13.574524,93.101377,13.574524,93.101377,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [46]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.178905,56.197529,49.0,49.0,51.536684,51.804421,7.178905,7.197529,7.178905,7.197529,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.856667,58.747223,46.0,46.0,165.293875,162.491692,12.856667,12.747223,12.856667,12.747223,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.587509,59.917583,48.0,48.0,158.445387,142.028796,12.587509,11.917583,12.587509,11.917583,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.667717,111.013390,49.0,49.0,3927.242751,3845.660488,62.667717,62.013390,62.667717,62.013390,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.164307,138.919632,111.0,111.0,793.228169,779.505849,28.164307,27.919632,28.164307,27.919632,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,105.564125,59.121834,122.0,122.0,270.137985,3953.663785,16.435875,62.878166,16.435875,62.878166,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.502686,70.912888,109.0,109.0,600.381599,1450.628133,24.502686,38.087112,24.502686,38.087112,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.537186,73.791260,136.0,136.0,507.924738,3869.927362,22.537186,62.208740,22.537186,62.208740,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


In [47]:
1+1

2

### Archivos enteros

In [ ]:
import os
path = "/home/school/math/final_analysis.csv"

In [ ]:
os.remove(URL + 'rl_csvs/ql_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)